# Sprawdzenie kernela 

Na początek sprawdź czy silnik wykonawczy Twojego notatnika to PySpark. 
Mógłby on być po prostu interpreterem Pythona, jednak wówczas zmienne kontekstu musielibyśmy tworzyć samodzielnie.

Sprawdź, czy obiekt kontekstu jest dostępny. W przypadku *Spark SQL* jest to `SparkSession`

In [1]:
spark

Dzięki powyższej informacji dowiedzieliśmy się nie tylko w jakim trybie został uruchomiony Spark obsługujący nasze polecenia, w jakiej jest wersji, ale także czy obsługuje funkcjonalność platformy Hive.

Dowiedz się także pod jakim użytkownikiem działamy w ramach tego notatnika.

In [2]:
%%sh 
whoami

root


Czas na nasze właściwe zadania. 

W razie potrzeby korzystaj z https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/index.html

# 20 Years of Games

**7. Zaczytaj do zmiennej gameInfosDF zawartość pliku ign.csv**

In [3]:
username = "mikolaj2002nowak" # UWAGA! ustaw zmienną username na poprawną wartość

gameInfosDF=spark.read.\
    option("inferSchema", "true").\
    csv(f"/user/{username}/ign.csv", header=True).cache()

**8. Wyświetl schemat zmiennej gameInfosDF**

In [4]:
gameInfosDF.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- score_phrase: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- score: double (nullable = true)
 |-- genre: string (nullable = true)
 |-- editors_choice: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- release_month: integer (nullable = true)
 |-- release_day: integer (nullable = true)



Możesz także po prostu przyglądnąć się jej kolumnom

In [5]:
gameInfosDF.columns

['_c0',
 'score_phrase',
 'title',
 'url',
 'platform',
 'score',
 'genre',
 'editors_choice',
 'release_year',
 'release_month',
 'release_day']

Zobaczmy też trzy pierwsze wiersze. Zróbmy to na kilka sposobów. 

* Na początek metoda `show()`

In [6]:
gameInfosDF.limit(3).show()

+---+------------+--------------------+--------------------+----------------+-----+----------+--------------+------------+-------------+-----------+
|_c0|score_phrase|               title|                 url|        platform|score|     genre|editors_choice|release_year|release_month|release_day|
+---+------------+--------------------+--------------------+----------------+-----+----------+--------------+------------+-------------+-----------+
|  0|     Amazing|LittleBigPlanet P...|/games/littlebigp...|PlayStation Vita|  9.0|Platformer|             Y|        2012|            9|         12|
|  1|     Amazing|LittleBigPlanet P...|/games/littlebigp...|PlayStation Vita|  9.0|Platformer|             Y|        2012|            9|         12|
|  2|       Great|Splice: Tree of Life|/games/splice/ipa...|            iPad|  8.5|    Puzzle|             N|        2012|            9|         12|
+---+------------+--------------------+--------------------+----------------+-----+----------+------------

Przetwarzane dane mogą być duże. Wyniki natomiast z reguły są znacznie mniejsze, to pozwala nam je (o ile znamy ich wielkość) przekonwertować do obiektów `pandas DataFrame` i dzięki temu przedstawić w przyjaźniejszej postaci.
* metoda `toPandas()`

In [7]:
gameInfosDF.limit(3).toPandas()

,_c0,score_phrase,title,url,platform,score,genre,editors_choice,release_year,release_month,release_day
0,0,Amazing,LittleBigPlanet PS Vita,/games/littlebigplanet-vita/vita-98907,PlayStation Vita,9.0,Platformer,Y,2012,9,12
1,1,Amazing,LittleBigPlanet PS Vita -- Marvel Super Hero E...,/games/littlebigplanet-ps-vita-marvel-super-he...,PlayStation Vita,9.0,Platformer,Y,2012,9,12
2,2,Great,Splice: Tree of Life,/games/splice/ipad-141070,iPad,8.5,Puzzle,N,2012,9,12


Za pomocą parametru konfiguracyjnego `spark.sql.repl.eagerEval.enabled` naszego kontekstu, również możemy 
ułatwić sobie wgląd w zawartość naszych wyników. Warto także ustawić parametr aby kontrolować liczbę pobieranych w ten sposób wierszy (tak, w razie niedoszacowania wyniku)
* parametr `spark.sql.repl.eagerEval.enabled`

In [8]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 3)
gameInfosDF

_c0,score_phrase,title,url,platform,score,genre,editors_choice,release_year,release_month,release_day
0,Amazing,LittleBigPlanet P...,/games/littlebigp...,PlayStation Vita,9.0,Platformer,Y,2012,9,12
1,Amazing,LittleBigPlanet P...,/games/littlebigp...,PlayStation Vita,9.0,Platformer,Y,2012,9,12
2,Great,Splice: Tree of Life,/games/splice/ipa...,iPad,8.5,Puzzle,N,2012,9,12


Wykorzystuj powyższe, aby móc podglądać uzyskiwane wyniki

 
**9. Na początek coś prostego. 
Wyświetl trzy najlepiej ocenione gry wydane w roku 2016 na platformę PC.**

In [9]:
from pyspark.sql.functions import col, lit

gameInfosDF.filter(col("platform") == "PC").where(col("release_year") == "2016").orderBy(col("score").desc()).limit(3).select("title","score")


title,score
Undertale,10.0
The Witness,10.0
Inside,10.0


**10. Określ dla każdej oceny opisowej (score_phrase) minimalną i 
maksymalną ocenę liczbową. Wyniki posortuj
rosnąco pod względem minimalnej oceny liczbowej.**

In [10]:
from pyspark.sql.functions import *
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 20)

gameInfosDF.groupBy("score_phrase").agg(max("score").alias("max"),min("score").alias("min")).select("score_phrase", "max", "min").orderBy(col("min").desc())


score_phrase,max,min
Masterpiece,10.0,10.0
Amazing,9.9,9.0
Great,8.9,8.0
Good,7.9,7.0
Okay,6.9,6.0
Mediocre,5.9,5.0
Bad,4.9,4.0
Awful,3.9,3.0
Painful,2.9,2.0
Unbearable,1.9,1.0


**11. To może coś trudniejszego. Wyznacz liczbę oraz średnią ocenę gier wydawanych w poszczególnych latach
począwszy od roku 2000 na poszczególne platformy. Nie analizuj wszystkich platform – ogranicz je tylko do
tych, dla których liczba wszystkich recenzji gier biorąc pod uwagę wszystkie lata przekroczyła 500.**

*Uwaga: Klasycznie odwołalibyśmy się do źródłowego zboru danych dwa razy. Raz aby wyznaczyć popularne platformy, a następnie aby wyznaczyć ostateczny wynik. 
Korzystając z funkcji analitycznych możesz to zadanie rozwiązać sięgając do źródłowych danych tylko raz.*

**Rozwiąż to zadanie na dwa sposoby:**

a. Za pomocą DataFrame API


In [11]:
from pyspark.sql.window import Window

platformsOver500 = gameInfosDF.\
  groupBy(expr("platform as platformName")).\
  agg(count(gameInfosDF._c0).alias("how_many")).\
  select("platformName", "how_many").\
  filter(col("how_many") > 500)

platformsOver500.\
  join(gameInfosDF, platformsOver500["platformName"] == gameInfosDF.platform).\
  groupBy("platform", "release_year").\
  agg(avg(col("score")).alias("avg"), count("_c0").alias("count")).\
  select("platform", "release_year", "avg", "count").\
  filter(col("release_year") > 1999).\
  orderBy(col("release_year"), col("platform"))


platform,release_year,avg,count
PC,2000,7.027819548872187,266
PlayStation,2000,6.438306451612902,248
PlayStation 2,2000,7.244444444444445,45
Game Boy Advance,2001,6.803947368421052,76
GameCube,2001,8.152941176470588,17
PC,2001,7.092444444444443,225
PlayStation,2001,6.26875,48
PlayStation 2,2001,7.197484276729558,159
Xbox,2001,7.787878787878788,33
Game Boy Advance,2002,6.69302325581395,172


b. Za pomocą SQL (po zarejestrowaniu źródeł danych jako tymczasowych perspektyw).

In [12]:
gameInfosDF.createOrReplaceTempView("gameinfos")

spark.sql("""
SELECT 
    gi1.release_year, 
    gi1.platform, 
    count(*), 
    avg(gi1.score)
FROM 
    gameInfos gi1 
JOIN 
    (SELECT platform, count(*) AS c 
     FROM gameInfos 
     GROUP BY platform 
     HAVING count(*) >= 500) gi2 
ON gi1.platform = gi2.platform 
GROUP BY gi1.release_year, gi1.platform
HAVING gi1.release_year >= 2000 
ORDER BY gi1.release_year, gi1.platform
""")


release_year,platform,count(1),avg(score)
2000,PC,266,7.0278195488721815
2000,PlayStation,248,6.438306451612904
2000,PlayStation 2,45,7.2444444444444445
2001,Game Boy Advance,76,6.803947368421052
2001,GameCube,17,8.152941176470588
2001,PC,225,7.092444444444446
2001,PlayStation,48,6.268750000000001
2001,PlayStation 2,159,7.19748427672956
2001,Xbox,33,7.78787878787879
2002,Game Boy Advance,172,6.693023255813954


**12. Jeśli masz swoją ulubioną serię gier (https://pl.wikipedia.org/wiki/Kategoria:Serie_gier_komputerowych)
zobacz jakie średnie oceny zdobyły poszczególne pozycje z tej serii. Wyniki posortuj chronologicznie.**

In [13]:
favorite_series = "FIFA"

gameInfosDF.filter(col("title").rlike(f".*{favorite_series}.*")).\
    groupBy("title", "release_year").\
    agg(avg("score").alias("avg_score"), count("*").alias("review_count")).\
    orderBy("release_year").\
    show()


+--------------------+------------+-----------------+------------+
|               title|release_year|        avg_score|review_count|
+--------------------+------------+-----------------+------------+
|     FIFA Soccer '96|        1996|              7.8|           1|
|FIFA Road to Worl...|        1997|              8.1|           2|
|      FIFA Soccer 64|        1997|              4.2|           1|
|            FIFA '99|        1998|             8.85|           2|
|FIFA 2000 Major L...|        1999|             8.95|           2|
|            FIFA '99|        1999|              9.0|           1|
|FIFA 2001: Major ...|        2000|              9.0|           3|
|FIFA 2000 Major L...|        2000|              2.0|           1|
|    FIFA Soccer 2002|        2001|              7.8|           3|
|    FIFA Soccer 2003|        2002|            8.025|           4|
| 2002 FIFA World Cup|        2002|              8.5|           4|
|    FIFA Soccer 2004|        2003|8.419999999999998|         

**13. (opcjonalne) Porównaj ze sobą gry wchodzące w skład wybranych serii gier wchodzących w skład 20
najlepszych serii wg Guinessa (lista z 2010 roku). W związku z tym, że gry nie są wydawane co roku, pogrupuj
dane w przedziały o długości 5 lat.**

In [14]:
selected_series = ["Super Mario", "The Legend of Zelda", "Pokemon"]

gameInfosDF.filter(col("title").rlike("|".join([f".*{series}.*" for series in selected_series]))).\
    withColumn("year_group", floor(col("release_year") / 5) * 5).\
    groupBy("year_group", "platform", "title").\
    agg(avg("score").alias("avg_score"), count("*").alias("review_count")).\
    orderBy("year_group", "platform", "title").\
    show()


+----------+----------------+--------------------+---------+------------+
|year_group|        platform|               title|avg_score|review_count|
+----------+----------------+--------------------+---------+------------+
|      1995|        Game Boy|Pokemon Blue Version|     10.0|           1|
|      1995|        Game Boy| Pokemon Red Version|     10.0|           1|
|      1995|        Game Boy|Pokemon Yellow: S...|     10.0|           1|
|      1995|  Game Boy Color|     Pokemon Pinball|      8.0|           1|
|      1995|  Game Boy Color|Super Mario Bros....|     10.0|           1|
|      1995|  Game Boy Color|The Legend of Zel...|     10.0|           1|
|      1995|     Nintendo 64|        Pokemon Snap|      7.8|           1|
|      1995|     Nintendo 64|Pokemon Stadium 2...|      8.1|           1|
|      1995|     Nintendo 64|Pokemon Stadium [...|      5.0|           1|
|      1995|     Nintendo 64|      Super Mario 64|      9.8|           1|
|      1995|     Nintendo 64|The Legen

 
# MondialDB – DataFrames

**14. Na początku do zmiennych `citiesDF`, `countriesDF` załaduj odpowiednio dane z plików
`mondial.cities.json`, `mondial.countries.json`**

In [15]:
citiesDF = spark.read.json(f"/user/{username}/mondial.cities.json").cache()
countriesDF = spark.read.json(f"/user/{username}/mondial.countries.json").cache()

**15. Zapoznaj się z ich strukturą. Zwróć uwagę na występujące typy array.**

In [16]:
citiesDF.printSchema()
countriesDF.printSchema()

root
 |-- _id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- elevation: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- location: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- other_names: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- population: long (nullable = true)
 |-- province: string (nullable = true)

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- area: double (nullable = true)
 |-- capital: string (nullable = true)
 |-- code: string (nullable = true)
 |-- gdp: double (nullable = true)
 |-- government: string (nullable = true)
 |-- independence: struct (nullable = true)
 |    |-- $date: string (nullable = true)
 |-- inflation: double (nullable = true)
 |-- name: stri

**16. Zanim zaczniesz realizować zadania, zapoznaj się ze funkcją `explode`, która nadaje się świetnie do pracy z tablicami i ich rozpłaszczania.**

**Przykładowe zapytanie:**

In [17]:
countriesDF.where("name = 'Poland'").\
            select(col("name"), explode(col("population")).alias("pop_in_years"))

name,pop_in_years
Poland,"{23929757, 1946}"
Poland,"{24613684, 1950}"
Poland,"{29405729, 1960}"
Poland,"{32749400, 1970}"
Poland,"{35061450, 1978}"
Poland,"{37878641, 1988}"
Poland,"{38230000, 2002}"
Poland,"{38167786, 2005}"
Poland,"{38153389, 2009}"
Poland,"{38501000, 2011}"


Zwróć uwagę także na inne funkcje z tej rodziny jak: `explode_outer`, `posexplode`, `posexplode_outer`
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html


Wszystkie zadania wykonaj korzystając *DataFrame API*. Nie korzystaj z SQL.

**17. Oblicz sumę ludności wszystkich Państw na rok 2010. 
W sytuacji gdy w danym kraju nie przeprowadzono
badania w roku 2010 wykorzystaj najnowsze z badań wcześniejszych.**

In [18]:
from pyspark.sql.window import Window

# Step 1: Rozwijanie danych populacji na poszczególne lata
countriesDF_with_pop = countriesDF.select(
    col("name"),
    explode(col("population")).alias("pop_in_years")  # Eksplozja tablicy populacji
)

# Step 2: Filtracja danych populacji na rok 2009 lub wcześniejsze
countries_pop_2009_or_before = countriesDF_with_pop.filter(col("pop_in_years.year") <= 2009)

# Step 3: Okno sortujące lata malejąco, aby znaleźć najnowsze dane do roku 2009
windowSpec = Window.partitionBy("name").orderBy(col("pop_in_years.year").desc())

# Step 4: Dodanie numeracji w ramach każdego kraju (wybór najnowszego roku <= 2009)
countries_pop_with_rownum = countries_pop_2009_or_before.withColumn(
    "row_num", 
    row_number().over(windowSpec)
)

# Step 5: Wybranie tylko najnowszego roku dla każdego kraju
latest_pop_for_2009 = countries_pop_with_rownum.filter(col("row_num") == 1)

# Step 6: Sumowanie populacji wszystkich krajów na rok 2009 (lub wcześniejszy, najnowszy rok)
total_population_2009 = latest_pop_for_2009.groupBy().agg(
    sum(col("pop_in_years.value")).alias("total_population_2009")
)

# Wynik
total_population_2009.show()


+---------------------+
|total_population_2009|
+---------------------+
|           6175723236|
+---------------------+



**18. Było ciężko? Nie wierzę.**

**Teraz już będzie z górki. Podaj nazwy i gęstość zaludnienia trzech krajów o największej gęstości zaludnienia w roku 2010.**

In [19]:
# Step 1: Rozwijanie danych populacji na poszczególne lata
countriesDF_with_pop = countriesDF.select(
    col("name"),
    col("area"),
    explode(col("population")).alias("pop_in_years")  # Eksplozja tablicy populacji
)

# Step 2: Filtracja danych populacji na rok 2010 lub wcześniejsze
countries_pop_2010_or_before = countriesDF_with_pop.filter(col("pop_in_years.year") <= 2010)

# Step 3: Okno sortujące lata malejąco, aby znaleźć najnowsze dane do roku 2010
windowSpec = Window.partitionBy("name").orderBy(col("pop_in_years.year").desc())

# Step 4: Dodanie numeracji w ramach każdego kraju (wybór najnowszego roku <= 2010)
countries_pop_with_rownum = countries_pop_2010_or_before.withColumn(
    "row_num", 
    row_number().over(windowSpec)
)

# Step 5: Wybranie tylko najnowszego roku dla każdego kraju
latest_pop_for_2010 = countries_pop_with_rownum.filter(col("row_num") == 1)

# Step 6: Obliczenie gęstości zaludnienia (populacja / powierzchnia) i sortowanie
density_top_20 = latest_pop_for_2010.withColumn(
    "density", 
    (col("pop_in_years.value") / col("area"))
).orderBy(col("density").desc()).limit(20)

# Wynik: top 20 krajów o największej gęstości zaludnienia w 2010
density_top_20.select("name", "pop_in_years.value", "area", "density").show()



+------------+---------+--------+------------------+
|        name|    value|    area|           density|
+------------+---------+--------+------------------+
|       Macao|   502113|    16.0|        31382.0625|
|      Monaco|    36845|     1.9|19392.105263157897|
|   Singapore|  5076700|   632.6| 8025.134366108125|
|   Hong Kong|  6864346|  1092.0| 6286.031135531135|
|     Melilla|    66411|    12.0|           5534.25|
|   Gibraltar|    28240|     6.5| 4344.615384615385|
|       Ceuta|    75694|    18.0| 4205.222222222223|
|  Gaza Strip|  1416963|   365.0| 3882.090410958904|
|     Bahrain|  1234596|   620.0| 1991.283870967742|
|    Holy See|      840|    0.44| 1909.090909090909|
|       Malta|   404962|   320.0|        1265.50625|
|     Bermuda|    64237|    53.3|1205.1969981238274|
|Sint Maarten|    37429|    34.0|1100.8529411764705|
|    Maldives|   325694|   300.0|1085.6466666666668|
|  Bangladesh|124355263|144000.0| 863.5782152777778|
|      Jersey|    87186|   117.0| 745.17948717


**19. Podaj trzy kraje o największym procencie ludności żyjącym w miastach powyżej 50 000 mieszkańców w roku
2010.**

In [20]:
# Step 1: Filtracja miast o populacji powyżej 50 000 mieszkańców
cities_with_pop = citiesDF.filter(col("population") > 50000)

# Grupowanie miast według kodów krajów i sumowanie ich populacji
city_population_by_country = cities_with_pop.groupBy("country").agg(
    sum("population").alias("urban_population")
)

# Step 2: Eksplozja danych populacji w krajach na poszczególne lata
countries_with_pop = countriesDF.select(
    col("name"),
    col("code"),
    explode(col("population")).alias("pop_in_years")
)

# Step 3: Filtracja danych populacji na rok 2010 lub wcześniejsze
countries_pop_2010_or_before = countries_with_pop.filter(col("pop_in_years.year") <= 2010)

# Step 4: Okno sortujące lata malejąco, aby znaleźć najnowsze dane do roku 2010
windowSpec = Window.partitionBy("name").orderBy(col("pop_in_years.year").desc())

# Dodanie numeracji w ramach każdego kraju (wybór najnowszego roku <= 2010)
countries_with_latest_pop = countries_pop_2010_or_before.withColumn(
    "row_num", 
    row_number().over(windowSpec)
).filter(col("row_num") == 1)

# Step 5: Połączenie danych o populacji miejskiej z danymi krajów
urban_vs_total_pop = city_population_by_country.join(
    countries_with_latest_pop,
    city_population_by_country["country"] == countries_with_latest_pop["code"]
).select(
    col("name").alias("country_name"),
    col("urban_population"),
    col("pop_in_years.value").alias("total_population"),
    (col("urban_population") / col("pop_in_years.value") * 100).alias("urban_percentage")
)

# Step 6: Sortowanie krajów według procentu ludności miejskiej i wyświetlenie top 20
urban_percentage_top_20 = urban_vs_total_pop.orderBy(col("urban_percentage").desc()).limit(20)

# Wynik: top 20 krajów z największym procentem ludności miejskiej
urban_percentage_top_20.show()

+--------------+----------------+----------------+------------------+
|  country_name|urban_population|total_population|  urban_percentage|
+--------------+----------------+----------------+------------------+
|       Melilla|           78476|           66411|118.16717110117301|
|         Ceuta|           82376|           75694|108.82764816233784|
|     Hong Kong|         7055071|         6864346|102.77848756458371|
|     Singapore|         5076700|         5076700|             100.0|
|       Curacao|          125000|          130627| 95.69231475881709|
|     Australia|        16915048|        19855288| 85.19165272243848|
|        Brunei|          279924|          332844| 84.10065976854023|
|Western Sahara|          183691|          222631|  82.5091743737395|
|        Taiwan|        16481172|        22876527| 72.04403010999003|
|       Bahamas|          248948|          353658| 70.39229990555847|
|   South Korea|        32332901|        48219172| 67.05403609999775|
|        Turkey|    

No cóż, dane dotyczące ludności w miastach są zapewne nowsze niż z 2010 roku.
Na marginesie, zarówno Melilla jak i Ceuta to hiszpańskie miasta, afrykańskie eksklawy położone na terytorium
Maroka. Oba liczą ponad 70 tyś mieszkańców i oba posiadają autonomię (uzyskaną jednocześnie w marcu 1995 roku)
dlatego znalazły się w naszym zestawieniu.
A co to takiego eksklawy i czy enklawa jest tym samym, to już możesz przeczytać samodzielnie np. tu:
https://pl.wikipedia.org/wiki/Eksklawa